In [1]:
def get_manager_nickname(string):
    '''
    Returns the nickname (human name) of the manager listed in the manager section of the YFPY Team object
    if it is read in as a string.

    Useful if the manager information has been read in from a CSV, for example.
    '''
    #TODO: redo this using regex or isalnum
    manager_info = string.split('"manager_id"')[1]
    manager_info_halves = manager_info.strip("})']").replace("\n","").replace(' ','').replace(':','').replace(',','').strip().split('"nickname"')
    nickname = manager_info_halves[1].replace('"','')
    nickname = nickname.capitalize()
    return nickname
    

In [2]:
def get_manager_id(string):
    '''
    Returns the ID of the manager listed in the manager section of the YFPY Team object
    if it is read in as a string.

    Useful if the manager information has been read in from a CSV, for example.
    '''
    manager_info = string.split('"manager_id"')[1]
    manager_info_halves = manager_info.strip("})']").replace("\n","").replace(' ','').replace(':','').replace(',','').strip().split('"nickname"')
    id = manager_info_halves[0]
    return id

In [3]:
def rename_standings_raw_data(dataframe):
    '''
    Rename commonly used features from standings data
    '''
    df = dataframe.rename({
                    'name': 'team_name',
                    'team_standings.rank' : 'standings_position',
                    'team_standings.outcome_totals.wins' : 'wins',
                    'team_standings.outcome_totals.losses' : 'losses',
                    'team_standings.outcome_totals.ties' : 'ties',
                    'team_standings.outcome_totals.percentage' : 'win_percentage',
                    'team_standings.points_for': 'points_for',
                    'team_standings.points_against' : 'points_against',
                    'team_standings.playoff_seed' : 'playoff_seed'
                }, axis = 1)
    return df

In [1]:
def map_teams_to_name(team_name):
    map = {
        'starr is born': 'Nick',
        '#1 patriots fan': 'Nick',
        'thug bungalow': 'Michael',
        'the ice cold bruschi' : 'Ricky',
        'who invited canada' : 'Peter',
        'the fishingtons' : 'Buddy',
        'the scientist' : 'Kyle',
        'gibbs me a break': 'Kyle',
        "victor cruisin": 'Frags',
        'big blue': 'Frags',
        'saquon these nuts' : 'Frags',
        "jadeveon's hit men" : 'Tim',
        'immortal duffmen': 'Pat',
        'thunder crotches': 'Jake',
        "kurtz' horror": 'Jones',
        'the twelve apostles': 'Jones',
        "lauren's legit team": 'Lauren',
        'supercam that ho': 'Lauren',
        "john brown's army": 'Eric',
        "we're back": 'Nate',
        'wereback': 'Nate',
        'the dirty diapers': 'Big Rick'
    }
    if team_name.lower() in map.keys():
        return map[team_name.lower()]
    else:
        return 'unknown'

In [5]:
def generate_season_matchup_data(yahoo_query, team_ids):
    '''
    Returns a ready-made dataframe containing information on each individual matchup for the season.

    Required parameters:
    --------------------
    - yahoo_query: YFPY yahoo query object that has been scoped to a specific season.
    - team_ids: (list) IDs assigned to each team by Yahoo, can be obtained from YFPY or Yahoo Fantasy API directly
    
    '''
    all_season_results = []
    for id in team_ids:
        print(f"Generating season matchup data for team ID {id}")
        for matchup in yahoo_query.get_team_matchups(id):
            #create and append a result for each matchup for a team before moving to the next
            result = {
                'week' : matchup.teams[0].team_points.week,
                'team_name' : matchup.teams[0].name,
                'season_id' : matchup.teams[0].managers[0].manager_id,
                'manager_name' : matchup.teams[0].managers[0].nickname,
                'opponent_team_name' : matchup.teams[1].name,
                'opponent_season_id' : matchup.teams[1].managers[0].manager_id,
                'opponent_name' : matchup.teams[1].managers[0].nickname,
                'projected_points_for' : matchup.teams[0].team_projected_points.total,
                'points_for' : matchup.teams[0].team_points.total,
                'projected_points_against' : matchup.teams[1].team_projected_points.total,
                'points_against' : matchup.teams[1].team_points.total
                }
            all_season_results.append(result)
    return pd.DataFrame(all_season_results)

In [7]:
def generate_roster_points_by_week(yfpy_yahoo_query):
    weekly_stats = {}
    position_counter = {
        'BN' : 0,
        'WR' : 0,
        'RB' : 0
    }
    for player in yfpy_yahoo_query:
        position = player.selected_position.position
    
        if position in position_counter.keys():
            position_counter[position]+=1
            position = position + str(position_counter[position])
            
        weekly_stats[position] = player.name.full
        weekly_stats[f"{position}_player_key"] = player.player_key
        weekly_stats[f"{position}_points"] = player.player_points.total
            
    for i in range(position_counter['BN']+1, 7):
        weekly_stats[f"BN{i}"] = None
        weekly_stats[f"BN{i}_points"] = None
    return weekly_stats
